In [54]:
"""
Example demonstrating the Robotics Toolbox adapter usage.

This example shows how to use the RTB adapter to:
1. Initialize the environment
2. Load/create robots and objects
3. Perform basic operations like forward kinematics, pose manipulation, etc.
"""

import numpy as np
import spatialmath as sm
import spatialgeometry as sg
from pandaSim.geometry.rtb_adapter import RoboticsToolboxAdapter
from pandaSim.geometry.utils import convert_pose
from pandaSim.geometry.utils import create_virtual_panda
# auto reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
panda_path = '../model/franka_description/robots/frankaEmikaPandaVirtual.urdf'
panda = create_virtual_panda(urdf_path=panda_path)

panda

ERobot: panda, 8 joints (RRRRRRRR), 1 gripper, 2 branches, geometry, collision
┌──────┬──────────────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│ link │         link         │ joint │   parent    │              ETS: parent to link               │
├──────┼──────────────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│    0 │ panda_link0          │       │ BASE        │ SE3()                                          │
│    1 │ panda_link1          │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│    2 │ panda_link2          │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│    3 │ panda_link3          │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│    4 │ panda_link4          │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│    5 │ panda_link5          │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(

In [56]:

def main():
    """Main example function demonstrating RTB adapter usage."""
    
    # Initialize the adapter
    print("Initializing RTB adapter...")
    adapter = RoboticsToolboxAdapter({
        "realtime": True,
        "rate": 100
    })
    
    # Create a virtual Panda robot (similar to the provided sample)
    print("Creating virtual Panda robot...")
    panda_path = '../model/franka_description/robots/frankaEmikaPandaVirtual.urdf'
    panda_virtual = create_virtual_panda(urdf_path=panda_path)
    
    # Add robot to the environment through adapter
    robot_dict = {
        "id": "panda",
        "entity": panda_virtual
    }
    adapter.entities["panda"] = panda_virtual
    adapter.env.add(panda_virtual)
    
    # Open the gripper fingers (from sample code)
    if hasattr(panda_virtual, 'grippers') and len(panda_virtual.grippers) > 0:
        panda_virtual.grippers[0].q = [0.035, 0.035]
    
    # Create a box object to grasp (from sample code)
    print("Creating box object...")
    box = adapter.add_primitive(
        "box",
        scale=[0.1, 0.07, 0.03],
        color='blue',
        T=sm.SE3(0.7, 0, 0.015)
    )
    
    # Set box transparency
    box_entity = box["entity"]
    if hasattr(box_entity, 'set_alpha'):
        box_entity.set_alpha(0.5)
    
    # Demonstrate some basic operations
    print("\nDemonstrating adapter operations:")
    
    # 1. Get joint positions
    print("Current joint positions:", adapter.get_joint_positions(robot_dict))
    
    # 2. Get robot pose
    robot_pose = adapter.forward_kinematics(robot_dict, link='panda_finger_virtual')
    print(robot_pose)
    print("Robot base pose (pq format):", robot_pose)
    
    # 3. Get box pose
    box_pose = adapter.get_pose(box)
    print("Box pose (pq format):", box_pose)
    
    # 4. Get box size
    box_size = adapter.get_size(box)
    print("Box size:", box_size)
    
    # 5. Get bounding box
    bbox = adapter.get_bbox(box)
    print("Box AABB min bounds:", bbox['min_bounds'])
    print("Box AABB max bounds:", bbox['max_bounds'])
    
    # 6. Forward kinematics
    try:
        ee_pose = adapter.forward_kinematics(robot_dict, output_type='pq')
        print("End-effector pose (pq format):", ee_pose)
    except Exception as e:
        print("Forward kinematics error:", e)
    
    # 7. Joint limits
    try:
        lower, upper = adapter.get_joint_limits(robot_dict)
        print("Joint limits - Lower:", lower)
        print("Joint limits - Upper:", upper)
    except Exception as e:
        print("Joint limits error:", e)
    
    # 8. Transform the box
    print("\nTransforming box...")
    transform = sm.SE3.Trans(0.1, 0.1, 0.05)  # Move box by [0.1, 0.1, 0.05]
    new_pose = adapter.transform(box, transform, apply=True, output_type='pq')
    print("New box pose after transformation:", new_pose)
    
    # 9. Create additional objects for demonstration
    print("\nCreating additional objects...")
    
    # Create a sphere
    sphere = adapter.add_primitive(
        "sphere",
        radius=0.05,
        color='red',
        pos=[0.5, 0.5, 0.5]
    )
    
    # Create a cylinder
    cylinder = adapter.add_primitive(
        "cylinder",
        radius=0.03,
        length=0.2,
        color='green',
        pos=[0.3, 0.3, 0.1]
    )
    
    print(f"Created {len(adapter.entities)} entities total")
    
    # Step the simulation to update visualizations
    print("\nStepping simulation...")
    for i in range(100):
        adapter.step_simulation(0.001)
    
    print("\nExample completed! Check the Swift visualizer window.")
    
    
    print("Closing environment...")



In [57]:

if __name__ == "__main__":
    main() 
    

Initializing RTB adapter...


Creating virtual Panda robot...
Creating box object...

Demonstrating adapter operations:
Current joint positions: [ 0.         -0.3         0.         -2.2         0.          2.
  0.78539816  0.        ]
[ 0.70622308  0.70622308 -0.03534061  0.03534061 -0.17821381  0.17821381
  0.13715108  0.13715108]
Robot base pose (pq format): [ 0.70622308  0.70622308 -0.03534061  0.03534061 -0.17821381  0.17821381
  0.13715108  0.13715108]
Box pose (pq format): [0.7   0.    0.015 1.    0.    0.    0.   ]


ValueError: Cannot extract vertices from this object type

In [77]:
"""Main example function demonstrating RTB adapter usage."""

# Initialize the adapter
print("Initializing RTB adapter...")
adapter = RoboticsToolboxAdapter({
    "realtime": True,
    "rate": 100
})

# Create a virtual Panda robot (similar to the provided sample)
print("Creating virtual Panda robot...")
panda_path = '../model/franka_description/robots/frankaEmikaPandaVirtual.urdf'
panda_virtual = create_virtual_panda(urdf_path=panda_path)

# Add robot to the environment through adapter
robot_dict = {
    "id": "panda",
    "entity": panda_virtual
}
adapter.entities["panda"] = panda_virtual
adapter.env.add(panda_virtual)

# Open the gripper fingers (from sample code)
if hasattr(panda_virtual, 'grippers') and len(panda_virtual.grippers) > 0:
    panda_virtual.grippers[0].q = [0.035, 0.035]

# Create a box object to grasp (from sample code)
print("Creating box object...")
box = adapter.add_primitive(
    "box",
    scale=[0.1, 0.07, 0.03],
    color='blue',
    T=sm.SE3(0.7, 0, 0.015)
)

# Set box transparency
box_entity = box["entity"]
if hasattr(box_entity, 'set_alpha'):
    box_entity.set_alpha(0.5)

# Demonstrate some basic operations
print("\nDemonstrating adapter operations:")

# 1. Get joint positions
print("Current joint positions:", adapter.get_joint_positions(robot_dict))

# 2. Get robot pose
robot_pose = adapter.forward_kinematics(robot_dict, link='panda_finger_virtual')
print("Robot pose (pq format):", robot_pose)


box['entity'].T = box['entity'].T * sm.SE3.RPY(np.pi/3, 0, np.pi/2)
adapter.env.step(0.001)

Initializing RTB adapter...


Creating virtual Panda robot...
Creating box object...

Demonstrating adapter operations:
Current joint positions: [ 0.         -0.3         0.         -2.2         0.          2.
  0.78539816  0.        ]
Robot pose (pq format): [ 0.70622308  0.70622308 -0.03534061  0.03534061 -0.17821381  0.17821381
  0.13715108  0.13715108]


In [89]:
cube = box['entity']
cylinder_entity = sg.Cylinder(radius=0.03, length=0.2)
import trimesh
box_mesh = trimesh.creation.box(extents=cube.scale)
cylinder_mesh = trimesh.creation.cylinder(radius=cylinder_entity.radius, height=cylinder_entity.length, transform=sm.SE3.RPY(np.pi/3, 0, np.pi/2).A )
box_mesh.apply_transform(cube.T)
box_mesh.extents, cube.scale,  cylinder_mesh.bounds

(array([0.06098076, 0.1       , 0.07562178]),
 array([0.1 , 0.07, 0.03]),
 array([[-0.10160254, -0.03      , -0.07598076],
        [ 0.10160254,  0.03      ,  0.07598076]]))

In [93]:
box_mesh.compute_stable_poses()[0][0]

cube.T = box_mesh.compute_stable_poses()[0][0]
adapter.env.step(0.001)


In [104]:

bbox = box_mesh.bounding_box_oriented

bbox.bounds[1] - bbox.bounds[0]

array([0.06098076, 0.1       , 0.07562178])

In [59]:
cylinder_entity = sg.Cylinder(radius=0.03, length=0.2)
sphere = sg.Sphere(radius=0.05)
box = sg.Box(scale=[0.1, 0.07, 0.03])
cylinder_entity.to_dict(), sphere.to_dict(), box.to_dict()



({'stype': 'cylinder',
  't': [0.0, 0.0, 0.0],
  'q': [0.0, 0.0, 0.0, 1.0],
  'v': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'color': 5000268,
  'opacity': 1.0,
  'radius': 0.03,
  'length': 0.2},
 {'stype': 'sphere',
  't': [0.0, 0.0, 0.0],
  'q': [0.0, 0.0, 0.0, 1.0],
  'v': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'color': 5000268,
  'opacity': 1.0,
  'radius': 0.05},
 {'stype': 'cuboid',
  't': [0.0, 0.0, 0.0],
  'q': [0.0, 0.0, 0.0, 1.0],
  'v': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'color': 5000268,
  'opacity': 1.0,
  'scale': [0.1, 0.07, 0.03]})

In [73]:
box.T = sm.SE3.RPY(np.pi/3, 0, np.pi/2)
box.to_dict()
adapter.env.step(0.001)
mesh = adapter.get_mesh(box)
mesh_size = mesh.bounds[1] - mesh.bounds[0]
mesh_size, box.scale, mesh.extents

(array([0.06098076, 0.1       , 0.07562178]),
 array([0.1 , 0.07, 0.03]),
 array([0.06098076, 0.1       , 0.07562178]))

In [53]:
adapter.add_primitive(
    "box",
    scale=[0.1, 0.07, 0.03],
    color='blue',
    T=sm.SE3.RPY(np.pi/3, 0, np.pi/2)
)


connection handler failed
Traceback (most recent call last):
  File "/home/vahid/envs/panda/lib/python3.12/site-packages/websockets/legacy/protocol.py", line 953, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vahid/envs/panda/lib/python3.12/site-packages/websockets/legacy/protocol.py", line 1023, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vahid/envs/panda/lib/python3.12/site-packages/websockets/legacy/protocol.py", line 1098, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vahid/envs/panda/lib/python3.12/site-packages/websockets/legacy/protocol.py", line 1155, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "/home/vahid/envs/panda/lib/python3.12/site-packages/websockets/legacy/framing.py", line 70, in read


KeyboardInterrupt: 

In [43]:
import trimesh

box_mesh = trimesh.creation.box(extents=box.scale)
cylinder_mesh = trimesh.creation.cylinder(radius=cylinder_entity.radius, height=cylinder_entity.length, transform=sm.SE3.RPY(np.pi/3, 0, np.pi/2).A )
box_mesh.apply_transform(np.eye(4))
box_mesh.extents, box.scale,  cylinder_mesh.bounds

(array([0.1 , 0.07, 0.03]),
 array([0.1 , 0.07, 0.03]),
 array([[-0.10160254, -0.03      , -0.07598076],
        [ 0.10160254,  0.03      ,  0.07598076]]))

In [44]:
cylinder_mesh.bounds

array([[-0.10160254, -0.03      , -0.07598076],
       [ 0.10160254,  0.03      ,  0.07598076]])

In [35]:
cylinder_entity.to_dict(), box.to_dict(), sphere.to_dict()

({'stype': 'cylinder',
  't': [0.0, 0.0, 0.0],
  'q': [0.0, 0.0, 0.0, 1.0],
  'v': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'color': 5000268,
  'opacity': 1.0,
  'radius': 0.03,
  'length': 0.2},
 {'stype': 'cuboid',
  't': [0.0, 0.0, 0.0],
  'q': [0.0, 0.0, 0.0, 1.0],
  'v': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'color': 5000268,
  'opacity': 1.0,
  'scale': [0.1, 0.07, 0.03]},
 {'stype': 'sphere',
  't': [0.0, 0.0, 0.0],
  'q': [0.0, 0.0, 0.0, 1.0],
  'v': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'color': 5000268,
  'opacity': 1.0,
  'radius': 0.05})

In [32]:
cylinder_mesh.bounds, cylinder_entity.length, cylinder_entity.radius

(array([[-0.03, -0.03, -0.1 ],
        [ 0.03,  0.03,  0.1 ]]),
 0.2,
 0.03)

In [8]:
# Create a sphere
sphere = adapter.add_primitive(
    "sphere",
    radius=0.05,
    color='red',
    pos=[0.5, 0.5, 0.5]
)
# Create a cylinder
cylinder_entity = adapter.add_primitive(
    "cylinder",
    radius=0.03,
    length=0.2,
    color='green',
    pos=[0.3, 0.3, 0.1]
)
# 3. Get box pose
box_pose = adapter.get_pose(box)
print("Box pose (pq format):", box_pose)

# 4. Get box size
box_size = adapter.get_size(box)
print("Box size:", box_size)

cylinder_entity.values()

Box pose (pq format): [0. 0. 0. 1. 0. 0. 0.]
Box size: [0.1  0.07 0.03]


dict_values(['3', 'cylinder', {'radius': 0.03, 'length': 0.2, 'color': 'green'}, cylinder,
[0.3 0.3 0.1]])

In [9]:
cylinder_entity.

SyntaxError: invalid syntax (922479030.py, line 1)

In [16]:
import trimesh
trimesh.creation.

<trimesh.Trimesh(vertices.shape=(66, 3), faces.shape=(128, 3))>

In [ ]:

# 5. Get bounding box
bbox = adapter.get_bbox(box)
print("Box AABB min bounds:", bbox['min_bounds'])
print("Box AABB max bounds:", bbox['max_bounds'])

# 6. Forward kinematics
try:
    ee_pose = adapter.forward_kinematics(robot_dict, output_type='pq')
    print("End-effector pose (pq format):", ee_pose)
except Exception as e:
    print("Forward kinematics error:", e)

# 7. Joint limits
try:
    lower, upper = adapter.get_joint_limits(robot_dict)
    print("Joint limits - Lower:", lower)
    print("Joint limits - Upper:", upper)
except Exception as e:
    print("Joint limits error:", e)

# 8. Transform the box
print("\nTransforming box...")
transform = sm.SE3.Trans(0.1, 0.1, 0.05)  # Move box by [0.1, 0.1, 0.05]
new_pose = adapter.transform(box, transform, apply=True, output_type='pq')
print("New box pose after transformation:", new_pose)

# 9. Create additional objects for demonstration
print("\nCreating additional objects...")

# Create a sphere
sphere = adapter.add_primitive(
    "sphere",
    radius=0.05,
    color='red',
    pos=[0.5, 0.5, 0.5]
)

# Create a cylinder
cylinder_entity = adapter.add_primitive(
    "cylinder",
    radius=0.03,
    length=0.2,
    color='green',
    pos=[0.3, 0.3, 0.1]
)

print(f"Created {len(adapter.entities)} entities total")

# Step the simulation to update visualizations
print("\nStepping simulation...")
for i in range(100):
    adapter.step_simulation(0.001)

print("\nExample completed! Check the Swift visualizer window.")

# Keep the environment running
try:
    input("Press Enter to close the environment...")
except KeyboardInterrupt:
    pass

print("Closing environment...")